# Income statement sensitivity analysis with python

__Sensitivity analysis is used to project values based on changes in an independent variable. This method is particularly useful to answer what if questions.__

We can apply sensitivity analysis in finance in order to analyse what if scenarios in any metrics or company financials that we are interested in. For example, we can analyse what will be the effect on net income if sales drop by 10%. Or we could simply have a range of values for sales and based on them, simulate potential effects on different financial ratios or company financial statements.

 Note that sensitivity analysis in finance has many other use cases. For example, we could predict the price of a stock by changing different variables affecting stock prices such as company earnings, debt ratio, etc.

Lets have a look at how to perform such a sensitivity analysis in a company income statement using Python.

We will perform an http request to retrieve Reliance income statement for the last year. You can get your free api key in financialmodelingprep and pass it as an argument in the url:

In [1]:
import pandas as pd
import requests

demo='7af0198f40a084eacb20ebf1e07a71d8'
stock = 'RELIANCE.NS'

IS = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{stock}?apikey={demo}').json()

IS2020 = IS[0]

__Then, we define our sensitivity values. In this case, we want to see the effect on the income statement if revenues grow by 5%. That will be our good outcome scenario. In addition, we will also check the effect of a 10% reduction of sales.__

In [2]:
growth = 1.05
negative_growth = 0.90

#create the dictionaries
sensititvity =  {}
sensititvity['last_year'] = {}
sensititvity['projection'] = {}
sensititvity['projection_bad'] = {}

We will add the three income statements, that is the actual and the two projected ones, into a Python dictionary. The dictionary will contain three nested dictionaries with each of the three income statements.

Next, we parse the API returned data and add each of the variables into the last_year dictionary by parsing the relevant keys.
 
Below is the code to extract the income statement line items and add them to our dictionary.

To keep it short, I will only extract a few of the income statement line items as per below code:

In [3]:
#latest available data
sensititvity['last_year']['revenue'] = IS2020['revenue']
sensititvity['last_year']['COGS'] = IS2020['costOfRevenue']
sensititvity['last_year']['grossProfit'] = IS2020['grossProfit']
sensititvity['last_year']['opinc'] = IS2020['operatingIncome']
sensititvity['last_year']['intexp'] = IS2020['interestExpense']
sensititvity['last_year']['netIncome'] = IS2020['netIncome']

__Having now the latest income statement for Reliance into the last_year dictionary, we can move on and calculate each of the line items as a percentage of sales. This percentage will be our key to determine each of the line items of the projected income statement other than sales.__

In [4]:
#item as a percentage of sales
sensititvity['last_year']['COGSpersales'] = sensititvity['last_year']['COGS'] /sensititvity['last_year']['revenue']
sensititvity['last_year']['grossProfitpersales'] = sensititvity['last_year']['grossProfit'] /sensititvity['last_year']['revenue']
sensititvity['last_year']['opincpersales'] = sensititvity['last_year']['opinc']/sensititvity['last_year']['revenue']
sensititvity['last_year']['intexppersales'] = sensititvity['last_year']['intexp']/sensititvity['last_year']['revenue']
sensititvity['last_year']['netIncomepersales']  = sensititvity['last_year']['netIncome']/sensititvity['last_year']['revenue']

# Projecting the Income Statement

Finally, we can start calculating our income statement projections. For the good projection, note that we calculate the increase of revenue by multiplying revenue by the growth rate. Then, the rest of the income statement items are calculated using the percentage of sales allocation that we computed above:

In [5]:
#good projection
sensititvity['projection']['revenue'] = sensititvity['last_year']['revenue'] * growth
sensititvity['projection']['COGS'] = sensititvity['last_year']['COGSpersales'] * sensititvity['projection']['revenue'] 
sensititvity['projection']['grossProfit'] = sensititvity['last_year']['grossProfitpersales'] * sensititvity['projection']['revenue'] 
sensititvity['projection']['opinc'] = sensititvity['last_year']['opincpersales'] * sensititvity['projection']['revenue'] 
sensititvity['projection']['intexp'] = sensititvity['last_year']['intexppersales'] * sensititvity['projection']['revenue'] 
sensititvity['projection']['netIncome'] = sensititvity['last_year']['netIncomepersales'] * sensititvity['projection']['revenue'] 

And we repeat the same steps in order to project the bad outcome income statement:

In [6]:
#bad projection
sensititvity['projection_bad']['revenue'] = sensititvity['last_year']['revenue'] * negative_growth
sensititvity['projection_bad']['COGS'] = sensititvity['last_year']['COGSpersales'] * sensititvity['projection_bad']['revenue'] 
sensititvity['projection_bad']['grossProfit'] = sensititvity['last_year']['grossProfitpersales'] * sensititvity['projection_bad']['revenue'] 
sensititvity['projection_bad']['opinc'] = sensititvity['last_year']['opincpersales'] * sensititvity['projection_bad']['revenue'] 
sensititvity['projection_bad']['intexp'] = sensititvity['last_year']['intexppersales'] * sensititvity['projection_bad']['revenue'] 
sensititvity['projection_bad']['netIncome'] = sensititvity['last_year']['netIncomepersales'] * sensititvity['projection_bad']['revenue'] 

# Reliance sensitivity analysis – overview

In the last step, we are going to represent the data in a Pandas DataFrame following the income statement structure. To do this, we are going to transform our three dictionaries into a Pandas DataFrame and perform some basic data manipulations.

First, lets create the Pandas DataFrame using the method pd.DataFrame.from_dict(). Note that we pass our dictionary as an argument:

In [7]:
sensitivity_analysis = pd.DataFrame.from_dict(sensititvity,orient='columns')
sensitivity_analysis

,last_year,projection,projection_bad
revenue,5.965540e+12,6.263817e+12,5.368986e+12
COGS,4.512470e+12,4.738094e+12,4.061223e+12
grossProfit,1.453070e+12,1.525724e+12,1.307763e+12
opinc,6.627100e+11,6.958455e+11,5.964390e+11
intexp,1.993500e+11,2.093175e+11,1.794150e+11
netIncome,3.935400e+11,4.132170e+11,3.541860e+11
COGSpersales,7.564227e-01,NaN,NaN
grossProfitpersales,2.435773e-01,NaN,NaN
opincpersales,1.110897e-01,NaN,NaN
intexppersales,3.341692e-02,NaN,NaN


By looking into above Pandas DataFrame, we can see that we still have some data clean up to do. For example, lets represent the amounts in millions by dividing each item by 1,000,000. Also, we do not need to keep the per sales percentage of each of income statement items. Therefore, we will drop all rows where the index contains per sales.

That should give us a cleaner overview of the projected income statements for Reliance:

In [8]:
#show in milions
sensitivity_analysis = sensitivity_analysis/1000000
sensitivity_analysis.reset_index(inplace=True)

sensitivity_analysis = sensitivity_analysis[~sensitivity_analysis["index"].str.contains('persales')]
sensitivity_analysis

,index,last_year,projection,projection_bad
0,revenue,5965540.0,6263817.0,5368986.0
1,COGS,4512470.0,4738093.5,4061223.0
2,grossProfit,1453070.0,1525723.5,1307763.0
3,opinc,662710.0,695845.5,596439.0
4,intexp,199350.0,209317.5,179415.0
5,netIncome,393540.0,413217.0,354186.0


 We have projected the income statement for Reliance based on a good and a bad outcome.<br>Alternatively, you could perform sensitivity analysis with multiple options or change other line items to project the effect on the income statement.